In [1]:
import polars as pl

import netaddr
import ipaddress
import functools

import os.path

FLUTTER_PARQUET = os.path.expanduser("~/src/ripe/rpki/rpki-flutter/data/*.parquet")

In [2]:
df = pl.read_parquet(FLUTTER_PARQUET).sort('pfx')

In [3]:
df

type,vp,gen_ts,capture_ts,asn,pfx,maxlen
str,str,i64,str,str,str,i64
"""S""","""rpki-validator.ripe.net""",1714521188,"""1714521303""","""AS13335""","""1.0.0.0/24""",24
"""S""","""routinator.labs.lacnic.net""",null,"""1714521304""","""AS13335""","""1.0.0.0/24""",24
"""S""","""rpki.bknix.co.th""",1714520960,"""1714521308""","""AS13335""","""1.0.0.0/24""",24
"""S""","""routinator.afrinic.net""",1714521352,"""1714521443""","""AS13335""","""1.0.0.0/24""",24
"""S""","""rpki-validator.ripe.net""",1714607585,"""1714607703""","""AS13335""","""1.0.0.0/24""",24
…,…,…,…,…,…,…
"""S""","""routinator.afrinic.net""",1717026348,"""1717027047""","""AS7018""","""99.96.0.0/13""",13
"""S""","""routinator.labs.lacnic.net""",null,"""1717113305""","""AS7018""","""99.96.0.0/13""",13
"""S""","""rpki-validator.ripe.net""",1717113230,"""1717113308""","""AS7018""","""99.96.0.0/13""",13


In [4]:
@functools.lru_cache(32*1024)
def prefix_first_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.first).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf

@functools.lru_cache(32*1024)
def prefix_last_to_bytes(inp: str) -> bytes:
    range = netaddr.IPNetwork(inp)
    buf = ipaddress.ip_address(range.last).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf


In [5]:
df = df.with_columns(
    prefix_first=df['pfx'].map_elements(prefix_first_to_bytes, return_dtype=bytes),
    prefix_last=df['pfx'].map_elements(prefix_last_to_bytes, return_dtype=bytes),
)

In [6]:
df.write_parquet("rpki-flutter-boundaries.parquet")